# H3 indicator calculation sql

H3 indicator calculation sqls using the LG methodology v0.1: https://docs.google.com/document/d/1s1rINj-YVDQ36Vu-Q3T1sc55hcVLROJclaaELRF9tck/edit#

List of indicators:

- Farm level indicators:
    - Water use
    - Unustainable water use
    - Land use
    - Greenhouse gas emission (in progress)

- Landscape level indicators
    - Deforestation risk 
    - Climate risk from land use
    - Biodiversity risk from land use change (in progress)


## Farm level:

### Water use

Impact (c,g) = volume (c,g) * blwft (c,g)

### Unsustainable water use

Same as above but where BWS >= 0.4

Impact (c,g) = volume (c,g) * blwft (c,g)* %water stress in location


### Land use

Impact (c,g) = Volume(c,g) * SUM(HA(c,pixel)) / SUM(Prod (c,pixel))

## Landscape level

### Deforestation risk

impact(c,g) = sum(defkernel def(pixel) * prod (pixel)) / sum(prod(pixel)) * land use impact / ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))

### Climate risk from land use

Impact (c,g) = (sum(carbon*def kernel * prod (pixel)) / sum(prod)) * land use impact / ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))

### Update 10/22: Include satelligence indicators:

#### 1. Satelligence deforestation:

#### 2. Satelligence deforestation risk:

In [1]:
# import libraries
from psycopg2.pool import ThreadedConnectionPool

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import json
import numpy as np

In [2]:
#set env
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value
        
#list(env.keys())

# set conexion to local ddbb
postgres_thread_pool = ThreadedConnectionPool(
    1, 
    50,
    host=env['API_POSTGRES_HOST'],
    port=env['API_POSTGRES_PORT'],
    user=env['API_POSTGRES_USERNAME'],
    password=env['API_POSTGRES_PASSWORD']
)

#get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

In [3]:
def psql(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        conn.rollback()
        print(e)

In [4]:
SQL_GET_H3_UNCOMPACT_GEO_REGION = """
CREATE OR REPLACE FUNCTION get_h3_uncompact_geo_region(geo_region_id uuid, h3_resolution int)
RETURNS TABLE (h3index h3index) AS 
$$
    SELECT h3_uncompact(geo_region."h3Compact"::h3index[], h3_resolution) h3index
    FROM geo_region WHERE geo_region.id = geo_region_id
$$ 
LANGUAGE SQL;
"""

SQL_SUM_H3_GRID_OVER_GEO_REGION = SQL_GET_H3_UNCOMPACT_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_h3_grid_over_georegion(
    geo_region_id uuid, 
    h3_resolution int,
    h3_table_name varchar, 
    h3_column_name varchar
)
RETURNS float AS
$$
    DECLARE
        sum float;
    BEGIN
        EXECUTE format(
            'SELECT sum(h3grid.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid ON h3grid.h3index = geo_region.h3index;
            ', h3_column_name, h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL = """
CREATE OR REPLACE FUNCTION get_h3_table_column_for_material(material_id uuid, h3_data_type material_to_h3_type_enum)
RETURNS TABLE (h3_table_name varchar, h3_column_name varchar, h3_resolution int) AS
$$
    SELECT h3_data."h3tableName", h3_data."h3columnName", h3_data."h3resolution"
    FROM h3_data
        INNER JOIN material_to_h3 ON material_to_h3."h3DataId" = h3_data.id
    WHERE material_to_h3."materialId" = material_id 
        AND material_to_h3.type = h3_data_type
    LIMIT 1;
$$
LANGUAGE SQL;
"""

SQL_SUM_MATERIAL_OVER_GEO_REGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL+"""
CREATE OR REPLACE FUNCTION sum_material_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        h3_table_name varchar;
        h3_column_name varchar;
        h3_resolution integer;
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO h3_table_name, h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        SELECT sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""



# Weighted mean total cropland area per pixel ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))
SQL_SUM_WEIGHTED_CROPLAND_AREA_ALL = """CREATE OR REPLACE FUNCTION sum_h3_weighted_cropland_area_all(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        agri_ha_h3_table_name varchar := 'h3_grid_spam2010v2r0_global_ha';
        agri_ha_h3_column_name varchar := 'spam2010V2R0GlobalHAcofA'; --update with new column name;
        chickens_ha_table_name varchar := 'h3_grid_chickens_glo_ha';
        chickens_ha_column_name varchar := 'glw32010ChickensHa';
        goats_ha_table_name varchar := 'h3_grid_goats_glo_lsu';
        goats_ha_column_name varchar := 'glw32010GoatsLsu';
        horses_ha_table_name varchar := 'h3_grid_horses_glo_ha';
        horses_ha_column_name varchar := 'glw32010HorsesHa';
        pigs_ha_table_name varchar := 'h3_grid_pigs_glo_ha';
        pigs_ha_column_name varchar := 'glw32010PigsHa';
        sheep_ha_table_name varchar := 'h3_grid_sheep_glo_ha';
        sheep_ha_column_name varchar := 'glw32010SheepHa';
        sum float;
    BEGIN
        --Get h3data table name and column for a given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        
        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * 
            (h3grid_all_ha.%I +  h3grid_chickens_ha.%I + h3grid_goats_ha.%I + h3grid_horses_ha.%I + h3grid_pigs_ha.%I + h3grid_sheep_ha.%I))
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_all_ha ON h3grid_all_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_chickens_ha ON h3grid_chickens_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_goats_ha ON h3grid_goats_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_horses_ha ON h3grid_horses_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_pigs_ha ON h3grid_pigs_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_sheep_ha ON h3grid_sheep_ha.h3index = geo_region.h3index;
                ', material_h3_column_name, agri_ha_h3_column_name, chickens_ha_column_name, 
                goats_ha_column_name, horses_ha_column_name, pigs_ha_column_name, sheep_ha_column_name, 
                material_h3_table_name, agri_ha_h3_table_name, chickens_ha_table_name, goats_ha_table_name,
                horses_ha_table_name, pigs_ha_table_name, sheep_ha_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;     
$$
LANGUAGE plpgsql;
"""
SQL_SUM_WEIGHTED_CROPLAND_AREA = """CREATE OR REPLACE FUNCTION sum_h3_weighted_cropland_area(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        agri_ha_h3_table_name varchar := 'h3_grid_spam2010v2r0_global_ha';
        agri_ha_h3_column_name varchar := 'spam2010V2R0GlobalHAcofA'; --update with new column name;
        sum float;
    BEGIN
        --Get h3data table name and column for a given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        
        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * h3grid_all_ha.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_all_ha ON h3grid_all_ha.h3index = geo_region.h3index
                ', material_h3_column_name, agri_ha_h3_column_name,material_h3_table_name, agri_ha_h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;     
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_deforestation_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        deforestation_h3_table_name varchar := 'h3_grid_deforestation_global'; --update with new deforestation table
        deforestation_h3_column_name varchar := 'hansenLoss2019';
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * h3grid_def.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_def ON h3grid_def.h3index = geo_region.h3index;
            ', material_h3_column_name, deforestation_h3_column_name, material_h3_table_name, deforestation_h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_carbon_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        carbon_h3_table_name varchar := 'h3_grid_carbon_global'; -- update this table with new one
        carbon_h3_column_name varchar := 'earthstat2000GlobalHectareEmissions';--update this table with new one
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material --
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum deforestation times carbon where material is produced --
        EXECUTE format(
                'SELECT sum(h3grid_mat.%I * h3grid_carbon.%I) --carbon has been already multiplied by the deforestation
                    FROM get_h3_uncompact_geo_region($1, $2) geo_region
                        INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_carbon ON h3grid_carbon.h3index = geo_region.h3index;', 
                material_h3_column_name, 
                carbon_h3_column_name,
                material_h3_table_name,
                carbon_h3_table_name
            )
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""
SQL_GET_PERCENTAGE_WATER_STRESS_AREA = """
CREATE OR REPLACE FUNCTION get_percentage_water_stress_area(
geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE
        aqueduct_h3_table_name varchar := 'h3_grid_aqueduct_global';
        aqueduct_h3_column_name varchar := 'bwsCat';
        h3_resolution integer := 6;
        percentage float;

    BEGIN
        EXECUTE format(
            'SELECT reduced.ws_area/ reduced.g_area as percentage
            FROM
                (SELECT 
                    sum(case when aqueduct.%I > 3 then 1 else 0 end) ws_area, count(aqueduct.%I) g_area
                FROM get_h3_uncompact_geo_region($1, $2) geo_region
                INNER JOIN %I aqueduct ON aqueduct.h3index = geo_region.h3index) reduced
            WHERE reduced.g_area > 0;',
            aqueduct_h3_column_name,
            aqueduct_h3_column_name,
            aqueduct_h3_table_name
                )
            USING geo_region_id, h3_resolution
            INTO percentage;
        RETURN percentage;
    END;
$$
LANGUAGE plpgsql;
"""


SQL_GET_BLWF_IMPACT = """
CREATE OR REPLACE FUNCTION get_blwf_impact(
    adminRegionId uuid,
    material_id uuid)
RETURNS float AS
$$
    DECLARE
        indicator_coeficient_table varchar := 'indicator_coefficient';
        indicator_coeficient_value varchar := 'value';
        value float;
    BEGIN
       
        EXECUTE format(
            'SELECT reduced.%I
                FROM (
                    SELECT ind_coef.%I
                    FROM %I ind_coef
                    WHERE (ind_coef."adminRegionId" = $1 AND ind_coef."materialId" = $2)
                    OR (ind_coef."materialId" = $2 AND ind_coef."adminRegionId" IS NULL )  
                    )reduced
                WHERE reduced.%I is not null;',
                indicator_coeficient_value,
                indicator_coeficient_value,
                indicator_coeficient_table,
                indicator_coeficient_value
                )
            USING adminRegionId, material_id
            INTO value;
        RETURN value;
    END;
$$
LANGUAGE plpgsql;
"""

In [41]:
# NEW QUERIES FOR SATELLIGENCE DATA
SQL_GET_H3_TABLE_COLUMN_FOR_INDICATOR = """
CREATE OR REPLACE FUNCTION get_h3_table_column_for_indicator(nameCode varchar)
RETURNS TABLE (h3_table_name varchar, h3_column_name varchar, h3_resolution int, h3_year int) AS
$$
    SELECT h3_data."h3tableName", h3_data."h3columnName", h3_data."h3resolution", h3_data."year"
    FROM h3_data
        INNER JOIN  "indicator" indicator ON  indicator.id = h3_data."indicatorId" 
    WHERE indicator."nameCode" = nameCode
    LIMIT 1;
$$
LANGUAGE SQL;
"""

SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+SQL_GET_H3_TABLE_COLUMN_FOR_INDICATOR+"""
CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_over_georegion(
    geo_region_id uuid,
    nameCode varchar,
    year int
)
RETURNS float AS
$$
    DECLARE 
        h3_table_name varchar;
        h3_column_name varchar;
        h3_resolution integer;
        h3_year integer;
        sum float;
    BEGIN
        -- Get h3data table name and column name for a given indicator
        SELECT * INTO h3_table_name, h3_column_name, h3_resolution, h3_year
        FROM get_h3_table_column_for_indicator(nameCode);
        
        -- Sum table column over region
        SELECT case when h3_year = year 
        then sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        else 0 end
        INTO sum;
        RETURN sum;
    END;      
$$
LANGUAGE plpgsql;
"""



In [48]:
#%%time
sourcing_locations = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    +SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    +SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION\
    + """
    SELECT
        id,
        sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
        sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
        --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
        sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
        sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon
        --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
        --get_blwf_impact("adminRegionId", "materialId") as raw_water
    FROM
        sourcing_location
    """, conn)

sourcing_locations

,id,production,harvested_area,raw_deforestation,raw_carbon
0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39
1,ea1556f9-27a4-4f0f-a308-bf2fc6a0ea3e,1.928499e+06,2.834290e+06,1.848665e+06,1.468524e+07
2,5e394935-c9e1-4957-8183-6377a229ae8a,7.144756e+01,4.013908e+01,4.183139e+01,2.770092e+01
3,6896d22f-5be4-4e32-900d-799a1aeba022,NaN,NaN,NaN,NaN
4,b476ee6c-e43d-4707-bfe3-f624e174d002,1.305452e+06,8.676576e+05,1.043980e+06,1.559114e+06
...,...,...,...,...,...
6027,2c6d03d7-4e05-49a8-a3e1-e7416ac6e684,1.948213e+06,8.767577e+04,1.948212e+06,3.337120e+07
6028,6bcf5406-6e41-40b7-a40c-09e8dac1c5a8,6.721587e+06,3.076254e+05,6.721584e+06,4.797538e+07
6029,3a775668-365b-4d3e-b8e8-9670923fbeb0,7.067528e+06,3.195916e+05,7.063033e+06,3.707557e+07
6030,78561c42-3006-4f21-a6c7-e7268193780e,9.837457e+06,4.722127e+05,9.740612e+06,3.985730e+07


In [60]:
sourcing_records = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CROPLAND_AREA \
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION \
    + SQL_GET_PERCENTAGE_WATER_STRESS_AREA \
    + SQL_GET_BLWF_IMPACT \
    + SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    + SQL_GET_H3_TABLE_COLUMN_FOR_INDICATOR \
    + """
    SELECT
        sr.id,
        sr.tonnage,
        sr.year,
        sum_satelligence_deforestation_over_georegion(sl."geoRegionId", 'DF_LUC_T', sr.year) as SatdeforetsationRisk2021,
        sum_satelligence_deforestation_over_georegion(sl."geoRegionId", 'GHG_LUC_T', sr.year) as Satdeforetsation2021,
        sl.id,
        sl.production,
        sl.harvested_area,
        --sl.weighted_all_ha,
        sl.raw_deforestation,
        sl.raw_carbon
        --sl.water_stress_perct
        --sl.raw_water        
    FROM
        sourcing_records sr
        INNER JOIN
            (
                SELECT
                    id,
                    "geoRegionId",
                    sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
                    sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
                    --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
                    sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
                    sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon
                    --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
                    --get_blwf_impact("adminRegionId", "materialId") as raw_water
                FROM
                    sourcing_location
            ) as sl
            on sr."sourcingLocationId" = sl.id
""", conn)

sourcing_records['land_per_ton'] = sourcing_records['harvested_area'] / sourcing_records['production']
##Weighted mean total cropland area
#sourcing_records['weighted_total_cropland_area'] = sourcing_records['weighted_all_ha'] / sourcing_records['production']
#sourcing_records['deforestation_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_deforestation'] / sourcing_records['weighted_total_cropland_area'], 0)
#sourcing_records['carbon_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_carbon'] / sourcing_records['weighted_total_cropland_area'], 0)
#
#
#sourcing_records['land_use'] = sourcing_records['land_per_ton'] * sourcing_records['tonnage']
#sourcing_records['deforestation'] = sourcing_records['deforestation_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['carbon_loss'] = sourcing_records['carbon_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['water_use'] = sourcing_records['raw_water'] * sourcing_records['tonnage']
#sourcing_records['unsustainable_water_use'] = sourcing_records['water_use'] * sourcing_records['water_stress_perct']
sourcing_records['satelligence_deforestation_risk'] = sourcing_records['satdeforetsationrisk2021']
sourcing_records['satelligence_deforestation'] = sourcing_records['satdeforetsation2021']
#sourcing_records.to_csv('test_impact_calc.csv')


sourcing_records

,id,tonnage,year,satdeforetsationrisk2021,satdeforetsation2021,id,production,harvested_area,raw_deforestation,raw_carbon,land_per_ton,satelligence_deforestation_risk,satelligence_deforestation
0,75d25991-7d15-4f36-9f9a-95dd127371e0,2400.0,2010,0.0,0.0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,1.864222e+21,0.0,0.0
1,e445629b-3e38-4516-aa9e-6023b2303255,2424.0,2011,0.0,0.0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,1.864222e+21,0.0,0.0
2,1a3d56e4-821e-448b-91b1-d22b35977927,2448.0,2012,0.0,0.0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,1.864222e+21,0.0,0.0
3,032eb005-fd3e-4fc5-9dee-cf922e7ab611,2472.0,2013,0.0,0.0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,1.864222e+21,0.0,0.0
4,7ca403f1-d5d5-444f-baed-a7ba2eea3148,2497.0,2014,0.0,0.0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,1.864222e+21,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18497,c6eba8ea-5148-4cc8-ad1c-dc531a487a85,5185.0,2020,0.0,0.0,2c6d03d7-4e05-49a8-a3e1-e7416ac6e684,1.948213e+06,8.767577e+04,1.948212e+06,3.337120e+07,4.500319e-02,0.0,0.0
18498,1b9a23dd-e2a2-45d1-91b9-887bcc4ca6c3,1352.5,2020,0.0,0.0,6bcf5406-6e41-40b7-a40c-09e8dac1c5a8,6.721587e+06,3.076254e+05,6.721584e+06,4.797538e+07,4.576679e-02,0.0,0.0
18499,97cbe7ab-37f4-4e48-afd6-d29e98b502b6,1016.0,2020,0.0,0.0,3a775668-365b-4d3e-b8e8-9670923fbeb0,7.067528e+06,3.195916e+05,7.063033e+06,3.707557e+07,4.521971e-02,0.0,0.0
18500,d4a2d079-37a2-47ab-a9a6-c31c0a84c97e,4089.0,2020,0.0,0.0,78561c42-3006-4f21-a6c7-e7268193780e,9.837457e+06,4.722127e+05,9.740612e+06,3.985730e+07,4.800150e-02,0.0,0.0
